## Validating FAISS vector database library

In [1]:
import faiss
import numpy as np

In [2]:
dimension = 6
index = faiss.IndexFlatL2(dimension)

In [3]:
numVectors = 5
vectors = np.random.rand(numVectors, dimension).astype('float32')

In [4]:
index.add(vectors)
print(f"Number of vectors in index: {index.ntotal}")

Number of vectors in index: 5


In [5]:
nlist = 5 # number of centroids
quantizer = faiss.IndexFlatL2(dimension)
indexIVF = faiss.IndexIVFFlat(quantizer, dimension, nlist)
if not indexIVF.is_trained:
    indexIVF.train(vectors)
indexIVF.add(vectors)

In [6]:
queryVector = np.random.rand(1, dimension).astype('float')
k = 5
distances, indices = index.search(queryVector, k)
print(f"Distances: {distances}")
print(f"Indices of nearest neighbors: {indices}")

Distances: [[0.27588114 0.42974606 0.7852927  0.84622496 1.0679461 ]]
Indices of nearest neighbors: [[1 4 0 2 3]]


In [7]:
print(f"Query: {queryVector}")

Query: [[0.8993358  0.35125344 0.57818523 0.93227463 0.49595493 0.75647527]]


In [8]:
print("Reference vectors")
print(indices)
for ind in indices[0]:
    referenceVector = vectors[ind]
    l2_distance = np.linalg.norm(referenceVector - queryVector)
    print(f"{referenceVector}: {l2_distance} compared to {distances[0][ind]}")

Reference vectors
[[1 4 0 2 3]]
[0.8428284  0.1854445  0.7689372  0.48647907 0.44912902 0.6676903 ]: 0.5252438582150282 compared to 0.4297460615634918
[0.40872866 0.38495746 0.8096351  0.96535826 0.84728193 0.85557365]: 0.6555501840848474 compared to 1.0679460763931274
[0.6832321  0.78050584 0.08805035 0.74802506 0.7927283  0.31820616]: 0.8861674623157997 compared to 0.2758811414241791
[0.3650586  0.7891033  0.4040466  0.50345266 0.8433206  0.94136655]: 0.9199048292732149 compared to 0.7852926850318909
[0.88845015 0.7723497  0.6887926  0.12524863 0.94789135 0.90725285]: 1.0334146973114673 compared to 0.8462249636650085


## Utilizing PyMuPDF to extract text from PDF for vector encoding

In [9]:
import fitz

In [32]:
import re

In [70]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [71]:
def dilate_array(arr, kernel_size=1):
    n = len(arr)
    result = np.zeros_like(arr)
    for i in range(n):
        start = max(0, i - kernel_size)
        end = min(n, i + kernel_size + 1)
        window = arr[start:end]
        result[i] = 1.0 if np.any(window == 1) else 0.0
    return result

def erode_array(arr, kernel_size=1):
    n = len(arr)
    result = np.zeros_like(arr)
    for i in range(n):
        start = max(0, i - kernel_size)
        end = min(n, i + kernel_size + 1)
        window = arr[start:end]
        result[i] = 1.0 if np.all(window == 1) else 0.0
    return result

In [72]:
# Empirically, this seems to work across a few different types of PDF files
# With more investigation, I would try turning this into a DP problem where up to some set percentage (maybe 15%) CAN be classified 
# as an "Introduction" section, thus allowing each page to bid for their slot and disincentivizing other pages.
def GetTableOfContentsEstimator(linksPerPage, pageCount, lookback=5):
    if pageCount <= lookback: return np.zeros(pageCount)
    # look for local maxima in sliding window over linksPerPage to identify candidates for 'table of contents' sections
    tableOfContentsEstimator = np.zeros(pageCount - lookback)
    averageLinks = np.mean(linksPerPage)
    for i in range(lookback, len(linksPerPage)):
        pastAverage = np.mean(linksPerPage[(i - lookback):i])
        current = linksPerPage[i]
        if current <= pastAverage:
            dropRatio = max((pastAverage - current), 1) / (pastAverage + 1e-6)
            magnitude = pastAverage / (pastAverage + averageLinks + 1e-6)
            frontBias = np.exp(-(5.0/pageCount) * (i - lookback)) # bias towards zero at the end of the array
            tableOfContentsEstimator[i - lookback] = dropRatio * magnitude * frontBias
        else:
            tableOfContentsEstimator[i - lookback] = 0.0
    binaryResult = np.where(tableOfContentsEstimator >= 0.5, 1, 0)
    # perform morphological closing with k=1
    binaryResult = dilate_array(erode_array(binaryResult, 1), 1)
    # return last position where binaryResult == 1, this is likely the final page of the introduction / table of contents
    for i in range(len(binaryResult) - 1, -1, -1):
        if binaryResult[i] == 1:
            return i + lookback

In [102]:
doc = fitz.open("Observability-Engineering.pdf")
pageCount = doc.page_count

# metadata
linksPerPage = []
blocksPerPage = []
linksToPage = {}
maxPageContentDimensions = [np.inf, np.inf, -np.inf, -np.inf]
# once-over to compute metadata before storing any paragraph info
for pageIndex in range(doc.page_count):
    page = doc.load_page(pageIndex)
    blocks = page.get_text("blocks")
    links = page.get_links()
    linksPerPage.append(len(links))
    blocksPerPage.append(len(blocks))
    for block in blocks:
        maxPageContentDimensions[:2] = np.minimum(maxPageContentDimensions[:2], block[:2])
        maxPageContentDimensions[2:] = np.maximum(maxPageContentDimensions[2:4], block[2:4])
    for link in links:
        if not 'page' in link: continue
        linkTo = link['page']
        linksToPage[linkTo] = linksToPage.get(linkTo, 0) + 1

# approximate which pages may be part of the introduction / table of contents
lastTableOfContentsPage = GetTableOfContentsEstimator(linksPerPage, pageCount, 5) # underlying array looks something like [1,1,1,1,0,0,0,0,0,0,0,0,0,0]
print(lastTableOfContentsPage)

# estimate which are the most likely content pages
if len(linksToPage) != 0:
    startingContentPage = np.min(list(linksToPage.keys()))
    endingContentPage = np.max(list(linksToPage.keys()))
    likelyContentPages = np.zeros(pageCount, dtype=int)
    likelyContentPages[startingContentPage:endingContentPage] = 1
else:
    likelyContentPages = np.ones(pageCount, dtype=int)
print(likelyContentPages)

# get metric data on blocks per page
averageBlocksPerPage = np.mean(blocksPerPage)
print(averageBlocksPerPage)

all_sentences = []
for pageIndex in range(doc.page_count):
    page = doc.load_page(pageIndex)
    blocks = page.get_text("blocks")
    if lastTableOfContentsPage and pageIndex < lastTableOfContentsPage:
        continue
    if not likelyContentPages[pageIndex]:
        if len(blocks) == 0 or len(blocks) < averageBlocksPerPage / 2:
            continue
    for block in blocks:
        paragraph_text = block[4].strip().replace("\n", " ")
        if paragraph_text.isnumeric() or len(paragraph_text) < 10: continue # hacky, but remove all very-short phrases as they're likely not substantial content
        all_sentences.append(paragraph_text)
doc.close()

17
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
8.489096573208723


In [103]:
print(all_sentences)

['observability.” While the criticisms aren’t without merit, the truth is that logs, metrics, and traces have long been the only concrete examples of telemetry people running real systems in the real world have had at their disposal to debug their systems, and it was thus inevitable that the narrative of the “three pillars” cropped up around them.', 'What resonates best with practitioners building systems in the real world isn’t abstract, airy-fairy ideas but an actionable blueprint that addresses and proposes solutions to pressing technical and cultural problems they are facing. This book man‐ ages to bridge the chasm that yawns between the philosophical tenets of observability and its praxis thereof, by providing a concrete (if opinionated) blueprint of what putting these ideas into practice might look like.', 'Instead of focusing on protocols or standards or even low-level representations of various telemetry signals, the book envisages the three pillars of observability as the tria

In [104]:
# group adjacent sentences depending on punctuation
def sentence_groups(lines):
    group = []
    for w in lines:
        strippedW = w.strip()
        parts = re.split(r'([.?!])', strippedW)
        for part in parts:
            if part == '.' or part == '?' or part == '!' and group:
                yield group
                group = []
            if not part.isnumeric() and len(part) >= 10:
                group.append(part)
    if group:
        yield group

In [105]:
def parse_sentences(text_list):
    full_text = ' '.join(text_list)
    sentences = nltk.tokenize.sent_tokenize(full_text)
    return [sent.strip() for sent in sentences if len(sent.strip()) > 10]

sentencesProcessed = parse_sentences(all_sentences)
for sentence in sentencesProcessed:
    print(sentence)
    print('----')

observability.” While the criticisms aren’t without merit, the truth is that logs, metrics, and traces have long been the only concrete examples of telemetry people running real systems in the real world have had at their disposal to debug their systems, and it was thus inevitable that the narrative of the “three pillars” cropped up around them.
----
What resonates best with practitioners building systems in the real world isn’t abstract, airy-fairy ideas but an actionable blueprint that addresses and proposes solutions to pressing technical and cultural problems they are facing.
----
This book man‐ ages to bridge the chasm that yawns between the philosophical tenets of observability and its praxis thereof, by providing a concrete (if opinionated) blueprint of what putting these ideas into practice might look like.
----
Instead of focusing on protocols or standards or even low-level representations of various telemetry signals, the book envisages the three pillars of observability as t

In [82]:
#simplifiedSentences = [' '.join(group) for group in sentence_groups(all_sentences)]
#for sentence in simplifiedSentences:
#    print(sentence)
#    print('---')

## Utilizing SBERT library to perform semantic encoding of chunks

In [106]:
from sentence_transformers import SentenceTransformer

In [107]:
import torch

In [108]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [109]:
corpus_embeddings = model.encode_document(sentencesProcessed, convert_to_tensor=True)

In [100]:
queries = [
    "What are the eligibility requirements for Medicare home health services?",
    "What qualifies a patient as homebound under CMS guidelines?",
    "What skilled nursing services are considered reasonable and necessary?",
    "What conditions must be met for physical therapy to be covered?",
    "What must be included in the physician’s plan of care?",
    "What are the requirements for the physician face-to-face encounter?",
    "What documentation is needed to prove medical necessity for skilled services?",
    "How should changes to the plan of care be documented during the episode?",
    "Under what conditions are home health aide services covered?",
    "What are the supervision requirements for home health aides?"
]

In [110]:
queries = [
    "What types of monitoring are needed for companies that run a large portion of their own systems on low-level hardware?",
    "Describe some advantages of test-driven development regarding upkeep of a software product.",
    "How can observability be coupled with a development effort to prevent rolling back deployments?"
]

In [111]:
top_k = 5
for query in queries:
    query_embedding = model.encode_query(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    similarity_scores = model.similarity(query_embedding, corpus_embeddings)[0]
    scores, indices = torch.topk(similarity_scores, k=top_k)

    print("\nQuery:", query)
    print("Top 5 most similar sentences in corpus:")

    for score, idx in zip(scores, indices):
        context = sentencesProcessed[idx-2:idx+2]
        print(f"(Score: {score:.4f})", context)



Query: What types of monitoring are needed for companies that run a large portion of their own systems on low-level hardware?
Top 5 most similar sentences in corpus:
(Score: 0.6790) ['Next, let’s look at ways those two approaches may coexist, depending on the needs of your organization.', 'Assessing Your Organizational Needs Just as systems and software are complementary, so too are the methods for understanding the way each behaves.', 'Monitoring best helps engineers understand system-level concerns.', 'Observability best helps engineers understand application- level concerns.']
(Score: 0.6660) ['Whether running systems on premises or with a cloud provider, this consideration is less about where that infrastructure lives and more about operational responsibility.', 'Whether you provision virtual machines in the cloud or administer your own databases on premises, the key factor is whether your team takes on the burden of ensuring infrastructure availability and performance.', 'Organiz